<a href="https://colab.research.google.com/github/housemLassoued/ML-deployment/blob/main/Calories_Burnt_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
df1=pd.read_csv("/content/calories.csv")
df2=pd.read_csv("/content/exercise.csv")

In [ ]:
df1.head()

,User_ID,Calories
0,14733363,231.0
1,14861698,66.0
2,11179863,26.0
3,16180408,71.0
4,17771927,35.0


In [ ]:
df2.head()

,User_ID,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,14733363,male,68,190.0,94.0,29.0,105.0,40.8
1,14861698,female,20,166.0,60.0,14.0,94.0,40.3
2,11179863,male,69,179.0,79.0,5.0,88.0,38.7
3,16180408,female,34,179.0,71.0,13.0,100.0,40.5
4,17771927,female,27,154.0,58.0,10.0,81.0,39.8


In [ ]:
df=pd.merge(df1,df2,on="User_ID")

In [ ]:
df.head()

,User_ID,Calories,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,14733363,231.0,male,68,190.0,94.0,29.0,105.0,40.8
1,14861698,66.0,female,20,166.0,60.0,14.0,94.0,40.3
2,11179863,26.0,male,69,179.0,79.0,5.0,88.0,38.7
3,16180408,71.0,female,34,179.0,71.0,13.0,100.0,40.5
4,17771927,35.0,female,27,154.0,58.0,10.0,81.0,39.8


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   User_ID     15000 non-null  int64  
 1   Calories    15000 non-null  float64
 2   Gender      15000 non-null  object 
 3   Age         15000 non-null  int64  
 4   Height      15000 non-null  float64
 5   Weight      15000 non-null  float64
 6   Duration    15000 non-null  float64
 7   Heart_Rate  15000 non-null  float64
 8   Body_Temp   15000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 1.0+ MB


In [ ]:
df['Gender']=df['Gender'].map({'female':0,'male':1})

In [ ]:
import plotly.express as px

corr_matrix = df.corr()


fig = px.imshow(corr_matrix, text_auto=True, color_continuous_scale='RdBu_r', title='Correlation BTW Features')

fig.show()

In [ ]:
df.drop(columns=['User_ID','Weight','Height','Gender'],inplace=True)

In [ ]:
df.shape

(15000, 5)

In [ ]:
df['Body_Temp'].unique()

array([40.8, 40.3, 38.7, 40.5, 39.8, 40.7, 40.4, 40.2, 37.8, 40. , 38.1,
       38.6, 39.2, 39.5, 41. , 37.7, 39.7, 39.3, 38.9, 41.2, 38.2, 39.6,
       38.3, 40.1, 39.9, 40.6, 38. , 38.5, 40.9, 39.1, 39. , 38.4, 41.4,
       39.4, 37.9, 41.1, 37.6, 38.8, 41.3, 37.5, 41.5, 37.4, 37.3, 37.2,
       37.1])

In [ ]:

df_melted = df.melt(var_name='Column', value_name='Value')


fig = px.box(df_melted, x='Column', y='Value', title='Boxplot des Colonnes')


fig.show()

In [ ]:

def remove_outliers_iqr(df, columns):
    df_cleaned = df.copy()

    for col in columns:
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR


        df_cleaned = df_cleaned[(df_cleaned[col] >= lower_bound) & (df_cleaned[col] <= upper_bound)]

    return df_cleaned


columns_to_check = df.columns
df_cleaned = remove_outliers_iqr(df, columns_to_check)



In [ ]:
df_cleaned_melted = df_cleaned.melt(var_name='Column', value_name='Value')


fig = px.box(df_cleaned_melted, x='Column', y='Value', title='Boxplot des Colonnes')


fig.show()

In [ ]:
df_cleaned.shape

(14626, 5)

In [ ]:
x=df_cleaned.drop(columns=['Calories'])
y=df_cleaned['Calories']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
train_preds = model.predict(x_train)
print('Training Error : ', mae(y_train, train_preds))

val_preds = model.predict(x_test)
print('Validation Error : ', mae(y_test, val_preds))


Training Error :  8.305718804999877
Validation Error :  8.416791274973386


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor()
model.fit(x_train,y_train)

RandomForestRegressor()

In [ ]:
train_preds = model.predict(x_train)
print('Training Error : ', mae(y_train, train_preds))

val_preds = model.predict(x_test)
print('Validation Error : ', mae(y_test, val_preds))


Training Error :  2.006504835843169
Validation Error :  5.197091218305504


In [ ]:
from xgboost import XGBRegressor
model=XGBRegressor()
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
train_preds = model.predict(x_train)
print('Training Error : ', mae(y_train, train_preds))

val_preds = model.predict(x_test)
print('Validation Error : ', mae(y_test, val_preds))


Training Error :  3.6923146980542403
Validation Error :  5.22364439091614


In [ ]:
from sklearn.svm import SVR
model=SVR()
model.fit(x_train,y_train)

SVR()

In [ ]:
train_preds = model.predict(x_train)
print('Training Error : ', mae(y_train, train_preds))

val_preds = model.predict(x_test)
print('Validation Error : ', mae(y_test, val_preds))


Training Error :  5.349282893060835
Validation Error :  5.3911637460474315
